In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
openai = OpenAI()

In [7]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

In [9]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThe website is a personal domain for Ed Donner, who is passionate about coding and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of **Nebula.io**, which focuses on using AI to improve talent discovery and management through proprietary LLMs. Previously, he founded AI startup **untapt**, which was acquired in 2021.\n\n### Notable Features:\n- **Outsmart**: A unique arena designed to challenge LLMs in diplomacy and strategic deception.\n- A mix of personal interests including DJing, amateur electronic music production, and engagement with the Hacker News community.\n\n### Recent Posts:\n- **December 21, 2024**: Welcome to SuperDataScientists!\n- **November 13, 2024**: Resources for mastering AI and LLM engineering.\n- **October 16, 2024**: Resources for transitioning from a software engineer to an AI data scientist.\n- **August 6, 2024**: Introduction to the Outsmart LLM Arena.\n\nThe website invites visitors to con

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Website Summary: Edward Donner

Edward Donner's website serves as a personal platform showcasing his interests and professional background in coding and machine learning, particularly in the realm of large language models (LLMs). 

## About Ed
- Ed is the co-founder and CTO of **Nebula.io**, a company focused on leveraging AI to enhance talent discovery and management.
- He has a background as the founder and CEO of **untapt**, an AI startup that was acquired in 2021.
- Ed enjoys amateur electronic music production and is an active participant in the Hacker News community.

## Key Features
- **Outsmart**: An experimental arena that allows LLMs to compete against each other, emphasizing diplomacy and strategic manipulation.

## Recent Updates
- **December 21, 2024**: Welcoming message aimed at SuperDataScientists.
- **November 13, 2024**: Announcement of resources for mastering AI and LLM engineering.
- **October 16, 2024**: Introduction of resources for transitioning from software engineering to AI data science.
- **August 6, 2024**: Further details on the **Outsmart LLM Arena**.

The website encourages connection and engagement with Ed through various platforms.

In [18]:
display_summary("https://cnn.com")

# CNN News Summary

CNN's website offers a comprehensive overview of the latest news and videos covering a wide range of topics including:

- **US and World News**: Reports on significant events such as the ongoing Ukraine-Russia conflict, protests in China, and accounts of natural disasters like California's wildfires.
- **Politics**: Insights into current political affairs, including updates on the 2024 elections and significant developments concerning figures like Donald Trump and special counsel Jack Smith.
- **Business and Finance**: Coverage of economic trends, market updates, and analyses of the fast-food industry changes.
- **Health**: Articles discussing health studies, mental well-being, and reports related to dietary impacts on longevity.
- **Environment and Climate**: Features on climate change issues, including discussions around rewilding efforts in Scotland and scientific insights related to Mars.
- **Sports and Entertainment**: Reports on major sporting events and celebrity news.
- **Exclusive Content**: In-depth analyses and investigative pieces on various social issues.

### Recent Highlights
- Protests in China escalated after a student incident, sparking public unrest.
- Strong winds and fire conditions remain a concern in California, with wildfire impacts being reported.
- A tragic event occurred in a Czech restaurant explosion resulting in loss of life.
- Ongoing discussions and negotiations regarding hostages in the Israel-Hamas conflict are being closely monitored.

CNN continues to provide up-to-date coverage of global events, driving discussions around critical issues facing society today.